<a href="https://colab.research.google.com/github/prupat/interactivedataviz/blob/main/ngrams_lab_2_Assignment_Prudence_Brou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import re

import nltk
# if you haven't downloaded punkt before, you only need to run the line below once
#nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize
nltk.download('punkt')

from nltk.util import bigrams
from nltk.lm.preprocessing import padded_everygram_pipeline

import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Part 1
An example of how ngrams are generated

In [11]:
# you will need to leverage the requests package
r = requests.get(r'https://www.gutenberg.org/files/1237/1237-0.txt')
father_goriot = r.text

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    father_goriot = father_goriot.replace(char, " ")

# check
print(father_goriot[:100])


ï»¿The Project Gutenberg EBook of Father Goriot, by Honore de Balzac    This eBook is for the use of


In [12]:
# remove the metadata at the beginning - this is slightly different for each book
father_goriot = father_goriot[983:]

#### Txt locally
If you'd rather use a file on your computer, here's the code -- you just need to save the text file in your local directory, and change the variables throughout.

The example is a report from the [Congressional Research Service](https://www.everycrsreport.com/files/2020-11-10_R45178_62d6238caecf6c02ddf495be33b3439f09eed744.pdf) on AI and National Security.

In [13]:
# this is simplified for demonstration
def sample_clean_text(text: str):
    # lowercase
    text = text.lower()

    # remove punctuation from text
    text = re.sub(r"[^\w\s]", "", text)

    # tokenize the text
    tokens = nltk.word_tokenize(text)

    # return your tokens
    return tokens

# call the function
sample_tokens = sample_clean_text(text = father_goriot)

# check
print(sample_tokens[:50])

['_', 'de', 'conflans', 'is', 'an', 'elderly', 'person', 'who', 'for', 'the', 'past', 'forty', 'years', 'has', 'kept', 'a', 'lodginghouse', 'in', 'the', 'rue', 'nuevesaintegenevieve', 'in', 'the', 'district', 'that', 'lies', 'between', 'the', 'latin', 'quarter', 'and', 'the', 'faubourg', 'saintmarcel', 'her', 'house', 'known', 'in', 'the', 'neighborhood', 'as', 'the', '_maison', 'vauquer_', 'receives', 'men', 'and', 'women', 'old', 'and']


In [14]:
# create bigrams from the sample tokens
my_bigrams = bigrams(sample_tokens)

# check
list(my_bigrams)[:10]

[('_', 'de'),
 ('de', 'conflans'),
 ('conflans', 'is'),
 ('is', 'an'),
 ('an', 'elderly'),
 ('elderly', 'person'),
 ('person', 'who'),
 ('who', 'for'),
 ('for', 'the'),
 ('the', 'past')]

# Part 2 - creating an ngram model


In [15]:
# 2 is for bigrams
n = 2
#specify the text you want to use
text = father_goriot


Now we are going to use an NLTK shortcut for preprocessing. This will:
* pad all of the sentences with `<s>` and `</s>` to train on sentence boundaries, too.
* create both unigrams and bigrams
* create a training set and a full vocab to train on

We need to give it a pre-tokenized text (we'll use nltk's tokenizer)

In [16]:
# step 1: tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# step 2: tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# step 3: convert each word to lowercase
tokenized_text = [[word.lower() for word in sent] for sent in tokenized_sentences]

#notice the sentence breaks and what the first 10 items of the tokenized text
print(tokenized_text[0])

['_', 'de', 'conflans', ')', 'is', 'an', 'elderly', 'person', ',', 'who', 'for', 'the', 'past', 'forty', 'years', 'has', 'kept', 'a', 'lodging-house', 'in', 'the', 'rue', 'nueve-sainte-genevieve', ',', 'in', 'the', 'district', 'that', 'lies', 'between', 'the', 'latin', 'quarter', 'and', 'the', 'faubourg', 'saint-marcel', '.']


Why tokenize sentences and words?
We want to be able to retain sentence boundaries to encode that, too.

In [18]:
# notice what the first 10 items are of the vocabulary
print(text[:10])

_ de Confl


In [19]:
# we imported this function from nltk
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [20]:
from nltk.lm import MLE
# we imported this function from nltk linear models (lm)
# it is for Maximum Likelihood Estimation

# MLE is the model we will use
lm = MLE(n)

In [21]:
# currently the vocab length is 0: it has no prior knowledge
len(lm.vocab)

0

In [22]:
# fit the model
# training data is the bigrams and unigrams
# the vocab is all the sentence tokens in the corpus

lm.fit(train_data, padded_sents)
len(lm.vocab)

10279

In [23]:
# inspect the model's vocabulary.
# be sure that a sentence you know exists (from tokenized_text) is in the
print(lm.vocab.lookup(tokenized_text[0]))

('_', 'de', 'conflans', ')', 'is', 'an', 'elderly', 'person', ',', 'who', 'for', 'the', 'past', 'forty', 'years', 'has', 'kept', 'a', 'lodging-house', 'in', 'the', 'rue', 'nueve-sainte-genevieve', ',', 'in', 'the', 'district', 'that', 'lies', 'between', 'the', 'latin', 'quarter', 'and', 'the', 'faubourg', 'saint-marcel', '.')


In [24]:
# see what happens when we include a word that is not in the vocab.
print(lm.vocab.lookup('then wear the gold hat iphone .'.split()))

('then', 'wear', 'the', 'gold', 'hat', '<UNK>', '.')


What did the model replace 'iphone' with?

Given that it didn't just return an "out of vocab" error, what does that mean about our model?

In [29]:
# how many times does eugene appear in the model?
print(lm.counts['eugene'])

# what is the probability of eugene appearing?
# this is technically the relative frequency of daisy appearing
lm.score('eugene')

190


0.0030181086519114686

In [30]:
# how often does (eugene, and) occur and what is the relative frequency?
print(lm.counts[['eugene']]['and'])
lm.score('and', 'eugene'.split())

9


0.022058823529411766

In [31]:
# what is the score of 'UNK'?

lm.score("<UNK>")

0.0

Does the relative frequency of 'UNK' change your assumption about how the model behaves?

How should we change our model to account for the fact the `<UNK>` words are not accounted for by the model?

Note: *Programmatically implementing this solution is beyond the scope of this course.*

## Generate text
We want to start our sentence with a word, and use that to predict all the words that come after that. We'll specify how long it should be.

There is a certain amount of randomness encoded into n-gram models. This prevents a model from becoming entirely deterministic. Maximum Likelihood Estimation without some degree of randomness will only produce the most likely result every time. Setting Random Seed means we will get the same result every time.

In [32]:
# generate a 20 word sentence starting with the word, 'eugene'

print(lm.generate(20, text_seed= 'eugene', random_seed=42))

['of', 'a', 'fortune', ',', 'those', 'that', 'was', 'a', 'life', ',', 'as', 'it', ',', 'and', 'saw', 'ten', 'francs', 'at', 'the', 'action']


This next code block is just to clean up the tokenized words and make them easier on human eyes. It is literally a detokenizer, which removes some extraneous text markup and reconciles some words back together.

In [33]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(lm, num_words, text_seed, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in lm.generate(num_words, text_seed=text_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [34]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='eugene', random_seed = 42)

'of a fortune, those that was a life, as it, and saw ten francs at the action'

Try a few more sentences, and try out another text. Once you are satisfied with what ngrams can (and cannot) do - post your code to your Github or another site.